<a href="https://colab.research.google.com/github/Its-Binto/AkinatorFB/blob/main/shhsdataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mne

import mne
!pip install wfdb
!pip install wfdb matplotlib
!pip install biosppy peakutils


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 71.6 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.1 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.0/158.0 kB 8.

In [2]:
import wfdb
record = wfdb.rdrecord('shhs1-200001', pn_dir='shhs')


NetFileNotFoundError: 404 Error: Not Found for url: https://physionet.org/content/shhs/

In [2]:
edf_path = '/content/shhs1-200001.edf'

raw = mne.io.read_raw_edf(edf_path, preload=True, verbose=False)

print("Channels:", raw.ch_names)
raw.plot(scalings='auto', duration=30, n_channels=10)


FileNotFoundError: [Errno 2] No such file or directory: '/content/shhs1-200001.edf'

In [ ]:
import pyedflib
import numpy as np
from biosppy.signals import ecg as biosppy_ecg
from scipy.stats import skew
from sklearn.preprocessing import StandardScaler
import os

# SHHS data paths (adjust based on your directory structure)
shhs_base_dir = '/path/to/shhs/data/'
edf_files = [
    'shhs1-200001.edf', 'shhs1-200002.edf',  # example SHHS filenames
    'shhs1-200003.edf', 'shhs1-200004.edf'
]

# Initialize lists
X_all, y_all = [], []

for edf_file in edf_files:
    print(f"\nProcessing record: {edf_file}")

    try:
        # Load EDF file
        edf_path = os.path.join(shhs_base_dir, edf_file)
        with pyedflib.EdfReader(edf_path) as record:
            # Get signal information
            signal_labels = record.getSignalLabels()
            fs = int(record.getSampleFrequency(0))  # assuming first channel's fs applies to all

            # Find required channels (SHHS specific names)
            ecg_index = None
            spo2_index = None
            for i, label in enumerate(signal_labels):
                if 'ECG' in label or 'EKG' in label:
                    ecg_index = i
                elif 'SaO2' in label or 'SpO2' in label:
                    spo2_index = i

            if ecg_index is None or spo2_index is None:
                print(f"Warning: Required channels not found in {edf_file}. Skipping...")
                continue

            # Load signals
            ecg = record.readSignal(ecg_index)
            spo2 = record.readSignal(spo2_index)

        # Load annotations (SHHS typically has separate annotation files)
        # This part will depend on how you've stored/accessed the apnea annotations
        # For demonstration, we'll create a dummy apnea_events dictionary
        # In practice, you'll need to parse the SHHS annotation files
        apnea_events = {}  # {sample_number: 'event_type'}
        # Example: You might parse from XML or other annotation files

        # R-peaks detection using BioSPPy
        biosppy_out = biosppy_ecg.ecg(signal=ecg, sampling_rate=fs, show=False)
        r_peaks = biosppy_out['rpeaks']

        # Sliding window parameters
        window_sec = 60
        step_sec = 30
        samples_per_window = window_sec * fs
        step_samples = step_sec * fs

        # Loop over windows
        for start in range(0, len(ecg) - samples_per_window + 1, step_samples):
            end = start + samples_per_window
            window_ecg = ecg[start:end]
            window_spo2 = spo2[start:end]

            # Extract per-second features
            time_series_features = []
            for i in range(0, samples_per_window, fs):
                ecg_1s = window_ecg[i:i+fs]
                spo2_1s = window_spo2[i:i+fs]

                # R-peaks in this second
                r_segment = r_peaks[(r_peaks >= start+i) & (r_peaks < start + i + fs)]
                rr = np.diff(r_segment) / fs

                # Heart rate features
                hr = 60 / rr.mean() if len(rr) > 0 else 0
                hrv = np.std(rr) if len(rr) > 0 else 0

                # SpO2 features
                spo2_mean = np.mean(spo2_1s)
                spo2_skew = skew(spo2_1s) if len(spo2_1s) > 1 else 0

                # Bradycardia/tachycardia flags
                is_brady = int(hr < 50)
                is_tachy = int(hr > 100)

                # RR irregularity
                rr_irreg = np.std(np.diff(rr)) if len(rr) > 2 else 0

                # Combine features
                features = [hr, hrv, spo2_mean, spo2_skew, rr_irreg, is_brady, is_tachy]
                time_series_features.append(features)

            # Labels - this depends on your apnea annotations
            # For SHHS, you might need to map sleep stages to apnea events
            # or use separate apnea annotations if available
            labels_in_window = [note for sample, note in apnea_events.items()
                               if start <= sample < end]
            apnea_labels = ['OA', 'CA', 'H', 'HA', 'CAA']  # same as your example
            label = [int(any(lbl in note for note in labels_in_window))
                    for lbl in apnea_labels]

            # Save features and label
            X_all.append(time_series_features)
            y_all.append(label)

    except Exception as e:
        print(f"Error processing {edf_file}: {e}")

# Convert to numpy arrays
X_all = np.array(X_all, dtype=np.float32)
y_all = np.array(y_all, dtype=np.float32)

# Normalize features
if len(X_all) > 0:
    X_all_flat = X_all.reshape(-1, X_all.shape[-1])
    scaler = StandardScaler().fit(X_all_flat)
    X_all = scaler.transform(X_all_flat).reshape(X_all.shape)

    # Dataset summary
    print("Apnea label distribution (total count per class):")
    print(np.sum(y_all, axis=0))

    print("\nCombined dataset shape:")
    print("X shape:", X_all.shape)
    print("y shape:", y_all.shape)
else:
    print("No data was processed - check your file paths and channel names")

In [ ]:
import wfdb
import matplotlib.pyplot as plt
import numpy as np

# Example loading from PhysioNet WFDB
record = wfdb.rdrecord('shhs1-200002', pb_dir=pb_dir, sampfrom=0, sampto=3000)  # ~30s if fs=100
ecg = record.p_signal[:, 0]  # Assuming ECG is first channel
fs = int(record.fs)


NameError: name 'pb_dir' is not defined